<a href="https://colab.research.google.com/github/koivunen/beamdefectdetect_dt2/blob/main/damage_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is a demo on detecting steel beam defects 

### Guide
 - Press **Run All** (Ctrl+F9) to install dependencies
    - The system will run the cells below until it gets killed
 - Press **Run All** (Ctrl+F9) again as we had to restart everything as part of the system installation process.
 - An upload dialog should pop up. Upload video named exactly **input.mp4**
 - Wait for the detection process to finish, this will take minutes/hours. Approx. one minute for five seconds of video.
 - A file should be offered as download with results!

In [ ]:
!pip install pyyaml==5.1
import torch
restart_runtime=False
try:
  import detectron2
except ImportError as e:
  restart_runtime=True
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
if restart_runtime:
  exit(0)

In [ ]:
!wget --no-clobber "https://seafile.utu.fi/f/cf91e875424b476cade8/?dl=1" -O weights.pth # beam_damage_detectron2_2022-2-23.pth

In [ ]:
from google.colab import files
uploaded = files.upload()
video_name = uploaded.keys()[0]

In [ ]:

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = 'weights.pth'
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
import cv2,tqdm
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.video_visualizer import VideoVisualizer

# From https://stackoverflow.com/a/62446532

video = cv2.VideoCapture(video_name)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

video_writer = cv2.VideoWriter('out2.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)

v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)

def runOnVideo(video, maxFrames):
 
    readFrames = 0
    while True:
        hasFrame, frame = video.read()
        if not hasFrame:
            break
        outputs = predictor(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        visualization = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)
        yield visualization
        readFrames += 1
        if readFrames > maxFrames:
            break

num_frames = 60*60 # one minute long at 60fps max

for visualization in tqdm.tqdm(runOnVideo(video, num_frames), total=num_frames):
    video_writer.write(visualization)

# Release resources
video.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
# Download the results
from google.colab import files
files.download(video_name+'.detect.mp4')